# Notes about the SAS's Course
# Predictive Modeling Using Logistic Regression (15.1)

This course covers predictive modeling using SAS/STAT software with emphasis on the LOGISTIC procedure. This course also discusses selecting variables and interactions, recoding categorical variables based on the smooth weight of evidence, assessing models, treating missing values, and using efficiency techniques for massive data sets. This notes are based in the course materials, some codes and images are copyrighted by Sas Institute. I made a Jupyter Notebook using JupiterLab with SAS University Edition.

In [6]:
/*Run this script to configurate the session*/

%let InicioCurso=/folders/myfolders/Cursos/EPMLR51/;
%include "&InicioCurso/setup.sas";

## Lesson 1: Understanding Predictive Modeling

### 1.1 Predictive Modeling Fundamentals

#### Goals of Predictive Modeling
* The main goal in predictive modeling is generalization, the ability to predict outcomes for new data.
* the aim to maximize predictive power as assessed by relevant metrics.
* In most business settings, the primary emphasis in predictive modeling is the empirical quality of the predictions. The secondary emphasis is on understanding the relationships between the predictor variables and the response variable. 
* In predictive modeling, we'll focus on fitting models that empirically maximize predictor power to meet the business needs for your organization.

#### Basic Steps of Predictive Modeling
* **The first step** in predictive modeling is building a model on historic data where the outcome variable value is known. 
* supervised classification: When the outcome variable is known and is discrete.The term supervised indicates that the class label is known for each case.
* The goal is to correctly classify cases into groups (or classes). The target variable is a class label. 
* When only two possible classes of outcomes exist, you refer to the target as binary. 
* A predictive model assigns a score to each case. When you have a binary target, each score measures the probability that the case belongs to a particular class. 

* **So, the second step** in predictive modeling is generalization. First, you build a predictive model on a subset of cases in which the target classification is known, and then you apply the predictive model to cases in which the target classification is unknown.

* Now let's look at the main tasks involved at each step of the predictive modeling process. For supervised classification: 
* * To prepare the input variables. missing values, dealing categorical inputs, redundancy, and performing variable screening.
* * Select the most predictive inputs and start fitting models. 

#### Applications of Predictive Modeling
A few typical examples are:

* target marketing.
* attrition prediction.
* credit scoring.
* fraud detection. 

Supervised classification also has fewer business-oriented uses. Image classification has applications in areas such as astronomy, nuclear medicine, and molecular genetics (McLachlan 1992; Ripley 1996; Hand 1997).

#### Activity: Exploring the Bank Data for the Target Marketing Project

In [7]:
data work.develop;
   set pmlr.develop;
run;

%global inputs;
%let inputs=ACCTAGE DDA DDABAL DEP DEPAMT CASHBK 
            CHECKS DIRDEP NSF NSFAMT PHONE TELLER 
            SAV SAVBAL ATM ATMAMT POS POSAMT CD 
            CDBAL IRA IRABAL LOC LOCBAL INV 
            INVBAL ILS ILSBAL MM MMBAL MMCRED MTG 
            MTGBAL CC CCBAL CCPURC SDB INCOME 
            HMOWN LORES HMVAL AGE CRSCORE MOVED 
            INAREA;

proc means data=work.develop n nmiss mean min max;
   var &inputs;
run;

proc freq data=work.develop;
   tables ins branch res;
run;

**Note:** To build the develop data set, the bank included all cases that have an Ins variable value of 1 and a representative sample of cases that have an Ins variable value of 0. This oversampling of the events increases the efficiency of the analysis because you are using a smaller sample and therefore have fewer cases to process. However, this oversampling also biases the results. You learn more about oversampling events, and how to adjust the model for it, later in the course. 

#### Practice: Exploring the Veterans' Organization Data Used in the Practices

In [8]:
data pmlr.pva(drop=control_number 
                   MONTHS_SINCE_LAST_PROM_RESP 
                   FILE_AVG_GIFT 
                   FILE_CARD_GIFT);
   set pmlr.pva_raw_data;
   STATUS_FL=RECENCY_STATUS_96NK in("F","L");
   STATUS_ES=RECENCY_STATUS_96NK in("E","S");
   home01=(HOME_OWNER="H");
   nses1=(SES="1");
   nses3=(SES="3");
   nses4=(SES="4");
   nses_=(SES="?");
   nurbr=(URBANICITY="R");
   nurbu=(URBANICITY="U");
   nurbs=(URBANICITY="S");
   nurbt=(URBANICITY="T");
   nurb_=(URBANICITY="?");
run;

2. To examine the contents of pmlr.pva, write a PROC CONTENTS step, submit it, and review the results. How many character variables are in the data set?

In [12]:
proc contents data=pmlr.pva;
run;

3. Write a PROC MEANS step that generates the following descriptive statistics for the numeric variables in the pmlr.pva data set: mean, number of missing values, maximum value, and minimum value. To specify only the numeric variables in the input data set, use the special SAS name list _NUMERIC_ in the VAR statement.

In [13]:
proc means data=pmlr.pva mean nmiss max min;
   var _numeric_;
run;

Variable,Mean,N Miss,Maximum,Minimum
TARGET_B TARGET_D MONTHS_SINCE_ORIGIN DONOR_AGE IN_HOUSE INCOME_GROUP PUBLISHED_PHONE MOR_HIT_RATE WEALTH_RATING MEDIAN_HOME_VALUE MEDIAN_HOUSEHOLD_INCOME PCT_OWNER_OCCUPIED PCT_MALE_MILITARY PCT_MALE_VETERANS PCT_VIETNAM_VETERANS PCT_WWII_VETERANS PEP_STAR RECENT_STAR_STATUS FREQUENCY_STATUS_97NK RECENT_RESPONSE_PROP RECENT_AVG_GIFT_AMT RECENT_CARD_RESPONSE_PROP RECENT_AVG_CARD_GIFT_AMT RECENT_RESPONSE_COUNT RECENT_CARD_RESPONSE_COUNT LIFETIME_CARD_PROM LIFETIME_PROM LIFETIME_GIFT_AMOUNT LIFETIME_GIFT_COUNT LIFETIME_AVG_GIFT_AMT LIFETIME_GIFT_RANGE LIFETIME_MAX_GIFT_AMT LIFETIME_MIN_GIFT_AMT LAST_GIFT_AMT CARD_PROM_12 NUMBER_PROM_12 MONTHS_SINCE_LAST_GIFT MONTHS_SINCE_FIRST_GIFT PER_CAPITA_INCOME STATUS_FL STATUS_ES home01 nses1 nses3 nses4 nses_ nurbr nurbu nurbs nurbt nurb_,0.2500000 15.6243444 73.4099732 58.9190506 0.0731984 3.9075434 0.4977287 3.3616560 5.0053967 1079.87 341.9702147 69.6989986 1.0290109 30.5739211 29.6032934 32.8524675 0.5044394 0.9311377 1.9839975 0.1901275 15.3653959 0.2308077 11.6854703 3.0431034 1.7305389 18.6680776 47.5705141 104.4257165 9.9797646 12.8583383 11.5878758 19.2088081 7.6209323 16.5841988 5.3671278 12.9018687 18.1911522 69.4820875 15857.33 0.0833161 0.2399339 0.5474912 0.3058022 0.1715362 0.0199773 0.0234359 0.2067417 0.1267809 0.2318294 0.2035928 0.0234359,0 14529 0 4795 0 4392 0 0 8810 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0,1.0000000 200.0000000 137.0000000 87.0000000 1.0000000 7.0000000 1.0000000 241.0000000 9.0000000 6000.00 1500.00 99.0000000 97.0000000 99.0000000 99.0000000 99.0000000 1.0000000 22.0000000 4.0000000 1.0000000 260.0000000 1.0000000 300.0000000 16.0000000 9.0000000 56.0000000 194.0000000 3775.00 95.0000000 450.0000000 997.0000000 1000.00 450.0000000 450.0000000 17.0000000 64.0000000 27.0000000 260.0000000 174523.00 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000,0 1.0000000 5.0000000 0 0 1.0000000 0 0 0 0 0 0 0 0 0 0 0 0 1.0000000 0 0 0 0 0 0 2.0000000 5.0000000 15.0000000 1.0000000 1.3600000 0 5.0000000 0 0 0 2.0000000 4.0000000 15.0000000 0 0 0 0 0 0 0 0 0 0 0 0 0


Is the proportion of events in the sample equal to the proportion of events in the population? (Hint: To find the proportion of events in the population, read the description of pmlr.pva_raw_data.) **different, target B (in the sample) is 0.25. In the population is 0.05 (review the explanation about de pvs_raw file)**


What is the average number of months since the last gift to the organization? **18.2**


How many numeric variables have missing values? **4**

4. Write a PROC FREQ step that generates frequency tables for the character variables in the pmlr.pva data set. To specify only the character variables in the input data set, add the special SAS name list _CHARACTER_ to the TABLES statement. To display the Number of Variable Levels table for each variable specified in the TABLES statement, include the NLEVELS option in the PROC FREQ statement.

In [14]:
proc freq data=pmlr.pva nlevels;
   tables _character_;
run;

5. Which character variable has the highest number of levels? **cluster_code**
6. How many dummy variables would need to be created for the character variable that has the highest number of levels? **54-1, 53**

###  1.2 Predictive Modeling Challenges
As a predictive modeler, you face a number of challenges. Some of your challenges are due to problems with the data. Other challenges are related to the analytic process of predictive modeling. Fortunately, techniques are available to overcome these challenges.

#### Data Challenges

* observational data, mixed measurement scales, high dimensionality, and rare target events. 

The data used for predictive modeling is typically **observational**. (It can also be called operational or opportunistic.) In other words, the data was collected for operational purposes (such as tax or accounting purposes) unrelated to statistical analysis (Huber 1997). Observational data is frequently massive, and often consists of millions of cases. You might have hundreds of input variables. Many are redundant or irrelevant. Observational data sets typically contain errors and missing values, so preparing data for predictive modeling is often difficult. 


When your data includes large numbers of input variables, you usually have to handle **mixed measurement** scales. The input variables might be interval, as in amounts. Others might be nominal, as in class names; ordinal, as in grades; or counts, as in number of boxes.Regression analysis requires inputs to be numeric, you need to convert the nominal input variable levels to numeric dummy variables, also known as design variables, as shown in this design matrix. A better solution is to collapse the levels when the avriables has many levels.

**high dimensionality**. The dimension refers to the number of input variables—actually, input degrees of freedom. Predictive modelers consider large numbers of input variables, typically hundreds. A high number of input variables gives you high dimensionality. You might think, "Isn't it good to have a lot of input variables?". The remedy for high dimensionality is dimension reduction. You ignore irrelevant and redundant variables without throwing out important ones. 

**rare target events**. That is, the event of interest is relatively rare compared to the non-event. For example, if the event represents less than 1% of the data set, it might be considered rare.When the target event is rare, a representative sample is unlikely to have enough target events to build a good predictive model. With rare events, the effective sample size for building a reliable prediction model is closer to 

**three times the number of event cases than to the nominal size of the data set (Harrell 1997)**. So a smaller sample data set could have the predictive potential of a massive data set. This means that it is possible to use a sample of the original data set to obtain (on average) a model of similar predictive power. However, this sample should be a non-representative sample.

One widespread strategy for predicting rare events is to build a model on a sample that disproportionately over-represents the event cases (for example, an equal number of events and non-events). This is typically done when the original data set is very large and the ratio of events to non-events is very small. This type of sampling is typically called oversampling or separate sampling and the sample is called a biased sample. 